In [33]:
import os
import numpy as np
import pickle
from tifffile import imread
from skimage import morphology
from skimage.morphology import remove_small_objects
from load_annotation_data import load_annotation_data
from classify_images import classify_images
from glob import glob
from PIL import Image
import time

In [19]:
#Inputs:
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024'
pthtest= r'C:\Users\Valentina\OneDrive - Johns Hopkins\Desktop\test png\testing images'
pthtestim = r'C:\Users\Valentina\OneDrive - Johns Hopkins\Desktop\test png\testing images\5x'

In [20]:
with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    nblack = data['nblack']
    nwhite = data['nwhite'] 
    classNames = data['classNames']

In [21]:
pthtestdata = os.path.join(pthtest, 'data py')
load_annotation_data(pthDL,pthtest,pthtestim)

 
Importing annotation data...
Image 1 of 2: SG_013_0013
 annotation data previously loaded
Image 2 of 2: SG_014_0016
 annotation data previously loaded


({'tile_name': ['00001.tif',
   '00002.tif',
   '00003.tif',
   '00004.tif',
   '00005.tif',
   '00006.tif',
   '00007.tif',
   '00008.tif',
   '00009.tif',
   '00010.tif',
   '00011.tif',
   '00012.tif',
   '00013.tif',
   '00014.tif',
   '00015.tif',
   '00016.tif',
   '00017.tif',
   '00018.tif',
   '00019.tif',
   '00020.tif',
   '00021.tif',
   '00022.tif',
   '00023.tif',
   '00024.tif',
   '00025.tif',
   '00026.tif',
   '00027.tif',
   '00028.tif',
   '00029.tif',
   '00030.tif',
   '00031.tif',
   '00032.tif',
   '00033.tif',
   '00034.tif',
   '00035.tif',
   '00036.tif',
   '00037.tif',
   '00038.tif',
   '00039.tif',
   '00040.tif',
   '00041.tif',
   '00042.tif',
   '00043.tif',
   '00044.tif',
   '00045.tif',
   '00046.tif',
   '00047.tif',
   '00048.tif',
   '00049.tif',
   '00050.tif',
   '00051.tif',
   '00052.tif',
   '00053.tif',
   '00054.tif',
   '00055.tif',
   '00056.tif',
   '00057.tif',
   '00058.tif',
   '00059.tif',
   '00060.tif',
   '00061.tif',
   '00062.t

In [17]:
pthclassifytest= classify_images(pthtestim, pthDL, color_overlay_HE=True, color_mask=False)

   
  Starting classification of image 1 of 2: SG_013_0013.png
  Image SG_013_0013.png already classified by this model
  Starting classification of image 2 of 2: SG_014_0016.png
  Image SG_014_0016.png already classified by this model
  Total time for classification: 0.0h 0.0m 9.503983497619629s


In [22]:
classNames = classNames[:-1]
numClasses = nblack-1

In [23]:
true_labels = []
predicted_labels = []

In [24]:
imlist = os.listdir(pthtestdata)

In [32]:
for folder in imlist:
    start  = time.time()
    pth_annotation_data = os.path.join(pthtestdata, folder)
    # print(pth_annotation_data)
    # annotation_folder = os.path.join(pthDL,img_name)
    if os.path.exists(os.path.join(pth_annotation_data, 'view_annotations.tif')) or os.path.exists(os.path.join(pth_annotation_data, 'view_annotations_raw.tif')):
        try:
        # Read the image and convert to double
            J0 = imread(os.path.join(pth_annotation_data, 'view_annotations.tif')).astype(np.double)
        except:
            J0 = imread(os.path.join(pth_annotation_data, 'view_annotations_raw.tif')).astype(np.double)
        
        # Read the imDL image
        imDL = Image.open(os.path.join(pthclassifytest, folder + '.png'))
        imDL_array = np.array(imDL)
        
        # Remove small pixels
        for b in range(0, int(J0.max())):
            tmp = J0 == b
            J0[J0 == b] = 0
            tmp = remove_small_objects(tmp.astype(bool), min_size=25, connectivity=2)
            J0[tmp] = b
        
        # Get true and predicted class at testing annotation locations
        L = np.where(J0 > 0)
        true_labels = np.concatenate((true_labels, J0[L]))
        predicted_labels = np.concatenate((predicted_labels, imDL_array[L]))


In [ ]:
predicted_labels = np.array(predicted_labels)
predicted_labels[predicted_labels == nblack] = nwhite

In [ ]:
# Normalize to the minimum number of pixels, rounded to nearest 1000
label_counts = np.bincount(true_labels)[1:]
label_percentages = (label_counts / label_counts.max() * 100).astype(int)
min_count = label_counts.min()

In [ ]:
print('Calculating total number of pixels in the testing dataset...')
for i, count in enumerate(label_counts):
    if label_percentages[i] == 100:
        print(f"  There are {count} pixels of {classNames[i]}. This is the most common class.")
    else:
        print(f"  There are {count} pixels of {classNames[i]}, {label_percentages[i]}% of the most common class.")
        

In [ ]:
 if 0 in label_counts:
    for i, count in enumerate(label_counts):
        if count == 0:
            print(f"\n No testing annotations exist for class {classNames[i]}.")
    raise ValueError("Cannot make confusion matrix. Please add testing annotations of missing class(es).")

In [ ]:
if min_count < 100:
    min_count = (min_count // 10) * 10
elif min_count < 1000:
    min_count = (min_count // 100) * 100
else:
    min_count = (min_count // 1000) * 1000

In [ ]:
for i, count in enumerate(label_counts):
    if count < 15000:
        print(f"\n  Only {count} testing pixels of {classNames[i]} found.")
        print("    We suggest a minimum of 15,000 pixels for a good assessment of model accuracy.")
        print("    Confusion matrix may be misleading.")

In [8]:
# Normalize Pixel counts
balanced_true_labels = []
balanced_predicted_labels = []
for label in np.unique(true_labels):
    indices = np.where(np.array(true_labels) == label)[0]
    selected_indices = np.random.choice(indices, min_count, replace=False)
    balanced_true_labels.extend(np.array(true_labels)[selected_indices])
    balanced_predicted_labels.extend(np.array(predicted_labels)[selected_indices])

In [ ]:
# Confusion matrix with equal number of pixels of each class
confusion_data= np.zeros((max(balanced_true_labels), max(balanced_predicted_labels)))
for true_label in range(1, max(balanced_true_labels) + 1):
    for pred_label in range(1, max(balanced_predicted_labels) + 1):
        confusion_data[true_label-1, pred_label-1] = np.sum(
            (np.array(balanced_true_labels) == true_label) & 
            (np.array(balanced_predicted_labels) == pred_label)
        )

confusion_data[np.isnan(confusion_data)] = 0

In [ ]:
# plot_confusion_matrix(confusion_data, classNames, pthDL)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(confusion_data, class_names, save_path=None):
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(confusion_data, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    
    # We want to show all ticks...
    ax.set(xticks=np.arange(confusion_data.shape[1]),
           yticks=np.arange(confusion_data.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=class_names, yticklabels=class_names,
           title='Confusion Matrix',
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    thresh = confusion_data.max() / 2.
    for i in range(confusion_data.shape[0]):
        for j in range(confusion_data.shape[1]):
            ax.text(j, i, format(confusion_data[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if confusion_data[i, j] > thresh else "black")

    fig.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()

# Usage
plot_confusion_matrix(confusion_data, classNames, 'confusion_matrix.png')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix_seaborn(confusion_data, class_names, save_path=None):
    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_data, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    if save_path:
        plt.savefig(save_path)
    plt.show()

# Usage
plot_confusion_matrix_seaborn(confusion_data, classNames, 'confusion_matrix_seaborn.png')